# Importamos las librerias

In [112]:
import os
import numpy as np
import glob
import shutil
import pandas as pd

In [113]:
import matplotlib.pyplot as plt

In [114]:
import tensorflow as tf

In [115]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [116]:
from sklearn.model_selection import train_test_split
import sklearn

# Cargamos la informacion

In [117]:
df = pd.read_csv(os.path.normpath('C:/Users/ctn24/PycharmProjects/ScoreModeling/PML_FPS.csv'), index_col=None)
df.columns

Index(['match_id', 'Date', 'League', 'club', 'Goals', 'Name', 'jersey_num',
       'pos_left', 'pos_top', 'height_cm', 'weight_kg', 'overall', 'potential',
       'weak_foot', 'pace', 'shooting', 'passing', 'dribbling', 'defending',
       'physic', 'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'defending_marking', 'defending_standing_tackle',
       'defending_sliding_tackle', 'int_work_att', 'int_work_def', 'left',
       'right', '

In [118]:
df.info(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7600 entries, 0 to 7599
Data columns (total 54 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   match_id                    7600 non-null   int64  
 1   Date                        7600 non-null   object 
 2   League                      7600 non-null   object 
 3   club                        7600 non-null   object 
 4   Goals                       7600 non-null   int64  
 5   Name                        7600 non-null   object 
 6   jersey_num                  7600 non-null   int64  
 7   pos_left                    7600 non-null   int64  
 8   pos_top                     7600 non-null   int64  
 9   height_cm                   7600 non-null   int64  
 10  weight_kg                   7600 non-null   int64  
 11  overall                     7600 non-null   int64  
 12  potential                   7600 non-null   int64  
 13  weak_foot                   7600 

In [119]:
matches_df = df.drop(['Date', 'League', 'club', 'Name', 'jersey_num'], axis=1)

In [120]:
matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7600 entries, 0 to 7599
Data columns (total 49 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   match_id                    7600 non-null   int64  
 1   Goals                       7600 non-null   int64  
 2   pos_left                    7600 non-null   int64  
 3   pos_top                     7600 non-null   int64  
 4   height_cm                   7600 non-null   int64  
 5   weight_kg                   7600 non-null   int64  
 6   overall                     7600 non-null   int64  
 7   potential                   7600 non-null   int64  
 8   weak_foot                   7600 non-null   int64  
 9   pace                        7600 non-null   int64  
 10  shooting                    7600 non-null   int64  
 11  passing                     7600 non-null   int64  
 12  dribbling                   7600 non-null   int64  
 13  defending                   7600 

## Convertimos las coordinadas de los jugadores en una "imagen"


In [121]:
feat_num = 45
cancha = np.zeros(( 9, 12, feat_num),dtype=int)
#cancha.tolist()
cancha.shape

(9, 12, 45)

In [122]:
prueba = cancha.copy()
prueba[1, 2, 0] = 1
prueba[:,:,0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [123]:
#dict_match_df[0].pos_left

In [124]:
#cancha[0, dict_match_df[0].pos_top, dict_match_df[0].pos_left] = 1
#cancha

In [125]:
#dict_match_df[0].drop(['pos_top','pos_left'], axis=1).info()

In [126]:
#feat_num = 3
#cancha = np.zeros((feat_num, 9, 12), dtype=int)
#cancha[0, dict_match_df[0].pos_top, dict_match_df[0].pos_left] = 2
#dict_match_df[0].drop(['pos_top','pos_left'], axis=1).head()
#cancha

In [127]:
matches_df.describe()

,match_id,Goals,pos_left,pos_top,height_cm,weight_kg,overall,potential,weak_foot,pace,...,mentality_penalties,defending_marking,defending_standing_tackle,defending_sliding_tackle,int_work_att,int_work_def,left,right,away,home
count,7600.000000,7600.000000,7600.000000,7600.0000,7600.000000,7600.000000,7600.000000,7600.000000,7600.000000,7600.000000,...,7600.000000,7600.000000,7600.000000,7600.000000,7600.000000,7600.000000,7600.000000,7600.000000,7600.000000,7600.000000
mean,189.500000,1.282895,5.515132,4.0000,182.210921,76.616974,75.225132,78.092237,3.122105,71.147500,...,60.860526,57.388026,61.415789,58.807368,1.305000,1.233158,0.248684,0.751316,0.500000,0.500000
std,109.703389,1.184504,3.701635,2.5537,6.689796,7.380812,5.186454,4.813899,0.659657,12.287363,...,13.826116,20.469810,20.251011,20.410816,0.553852,0.581873,0.432279,0.432279,0.500033,0.500033
min,0.000000,0.000000,0.000000,0.0000,162.000000,60.000000,12.000000,62.000000,1.000000,23.000000,...,21.000000,13.000000,17.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,94.750000,0.000000,2.000000,2.0000,178.000000,72.000000,72.000000,75.000000,3.000000,66.000000,...,51.000000,37.000000,45.000000,40.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000
50%,189.500000,1.000000,5.500000,4.0000,183.000000,76.000000,75.000000,78.000000,3.000000,73.000000,...,62.000000,66.000000,71.000000,67.000000,1.000000,1.000000,0.000000,1.000000,0.500000,0.500000
75%,284.250000,2.000000,9.000000,6.0000,187.000000,82.000000,79.000000,82.000000,3.000000,79.000000,...,71.000000,74.000000,77.000000,75.000000,2.000000,2.000000,0.000000,1.000000,1.000000,1.000000
max,379.000000,8.000000,11.000000,8.0000,201.000000,95.000000,88.000000,90.000000,5.000000,96.000000,...,95.000000,87.000000,90.000000,86.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000


In [128]:
matches_df.groupby(['match_id','home']).mean()

Goals  pos_left  pos_top  height_cm  weight_kg  overall  \
match_id home                                                            
0        0.0     0.0       9.3      4.0      182.7       76.6     69.2   
         1.0     3.0       2.1      4.0      178.9       70.0     78.9   
1        0.0     2.0       8.4      4.0      183.1       77.1     76.9   
         1.0     1.0       2.8      4.0      182.0       75.9     74.7   
2        0.0     1.0       9.3      4.0      183.3       77.7     74.3   
...              ...       ...      ...        ...        ...      ...   
377      1.0     1.0       2.1      4.0      184.6       78.2     76.7   
378      0.0     2.0       9.3      4.0      185.1       77.9     73.1   
         1.0     2.0       2.1      4.0      180.7       79.1     72.6   
379      0.0     1.0       8.4      4.0      181.2       76.1     83.2   
         1.0     2.0       2.0      4.0      181.7       75.4     74.1   

               potential  weak_foot  pace  shooting  ...  mentality_vision  \
match_id home                                        ...                     
0        0.0        71.3        3.2  70.1      57.5  ...              63.3   
         1.0        82.2        3.2  70.9      64.2  ...              69.2   
1        0.0        80.1        3.2  74.3      64.8  ...              69.6   
         1.0        77.7        2.9  71.5      60.8  ...              62.7   
2        0.0        77.7        3.1  71.2      60.4  ...              61.1   
...                  ...        ...   ...       ...  ...               ...   
377      1.0        80.9        3.2  73.7      65.4  ...              70.3   
378      0.0        75.4        3.0  70.1      57.4  ...              58.5   
         1.0        75.5        3.0  69.3      61.4  ...              62.3   
379      0.0        84.3        3.3  77.8      65.0  ...              71.4   
         1.0        76.5        2.9  71.4      60.9  ...              65.4   

               mentality_penalties  defending_marking  \
match_id home                                           
0        0.0                  53.4               54.6   
         1.0                  65.5               57.0   
1        0.0                  64.4               52.9   
         1.0                  62.0               54.6   
2        0.0                  60.4               56.6   
...                            ...                ...   
377      1.0                  65.6               52.5   
378      0.0                  59.1               56.2   
         1.0                  66.1               57.9   
379      0.0                  64.4               58.2   
         1.0                  64.9               59.6   

               defending_standing_tackle  defending_sliding_tackle  \
match_id home                                                        
0        0.0                        55.3                      54.5   
         1.0                        62.5                      59.8   
1        0.0                        61.2                      55.8   
         1.0                        61.1                      58.6   
2        0.0                        61.6                      61.7   
...                                  ...                       ...   
377      1.0                        59.2                      53.8   
378      0.0                        59.6                      56.1   
         1.0                        63.2                      60.1   
379      0.0                        59.7                      60.4   
         1.0                        60.5                      60.0   

               int_work_att  int_work_def  left  right  away  
match_id home                                                 
0        0.0            1.3           1.4   0.4    0.6   1.0  
         1.0            1.6           1.3   0.2    0.8   0.0  
1        0.0            1.4           0.9   0.2    0.8   1.0  
         1.0            1.2           1.1   0.4    0.6   0.0  
2     

In [129]:
matches_df.groupby('home').mean()

,match_id,Goals,pos_left,pos_top,height_cm,weight_kg,overall,potential,weak_foot,pace,...,mentality_vision,mentality_penalties,defending_marking,defending_standing_tackle,defending_sliding_tackle,int_work_att,int_work_def,left,right,away
home,,,,,,,,,,,,,,,,,,,,,
0.0,189.5,1.092105,8.730789,4.0,182.219737,76.607632,75.183947,78.058158,3.121316,71.109474,...,64.457105,60.707632,57.623158,61.648421,59.016579,1.305263,1.233158,0.244737,0.755263,1.0
1.0,189.5,1.473684,2.299474,4.0,182.202105,76.626316,75.266316,78.126316,3.122895,71.185526,...,64.748684,61.013421,57.152895,61.183158,58.598158,1.304737,1.233158,0.252632,0.747368,0.0


In [130]:
matches_arr = matches_df.drop(columns='match_id').to_numpy().reshape((380,20,48))

In [131]:
#matches_df.to_numpy().reshape((380,20,49))

In [132]:
#match_id_ls = matches_df['match_id'].unique().tolist()
#idk_lst = []
#for match_id in match_id_ls:
#    idk_lst.append(matches_df[matches_df.match_id==match_id].drop(columns='match_id'))
#idk_lst

In [133]:
#matches_arr = np.array(idk_lst, int)
#matches_arr.shape
#matches_arr[0,0,:]

In [134]:
matches_arr[1,:,2]

array([3., 4., 0., 1., 8., 7., 4., 5., 1., 7., 2., 3., 8., 0., 3., 5., 0.,
       8., 6., 5.])

In [135]:
#matches_arr[0,:,:]

#### Le quito los primeros 3 features, Goles, pos left y pos top

In [136]:
left_cord = matches_arr[0,:,1]
top_cord = matches_arr[0,:,2]
goals_arr = matches_arr[0,:,0]

matches_arr_cln = matches_arr.copy()
matches_arr_cln = matches_arr_cln[:,:,3:]
matches_arr_cln.shape

(380, 20, 45)

In [137]:
matches_arr_cln[0,0,:]

array([175.,  64.,  80.,  80.,   4.,  43.,  72.,  83.,  79.,  74.,  66.,
        79.,  67.,  69.,  87.,  76.,  79.,  82.,  81.,  85.,  83.,  46.,
        41.,  66.,  81.,  83.,  78.,  65.,  67.,  62.,  75.,  73.,  83.,
        73.,  80.,  85.,  73.,  72.,  70.,   1.,   1.,   0.,   1.,   0.,
         1.])

In [138]:
matches_arr_cln[0,:,0]

array([175., 168., 183., 178., 178., 198., 178., 175., 171., 185., 178.,
       186., 186., 178., 183., 183., 193., 177., 179., 184.])

In [139]:
top_cord.shape, left_cord.shape

((20,), (20,))

In [140]:
top_cord.astype(int)

array([6, 1, 8, 2, 0, 5, 3, 7, 4, 4, 8, 0, 3, 4, 5, 3, 5, 4, 0, 8])

In [141]:
cancha[top_cord.astype(int),left_cord.astype(int), :].shape

(20, 45)

In [142]:
cancha.shape

(9, 12, 45)

In [143]:
matches_arr_cln[0,:,:].T.shape

(45, 20)

#### No me acuerdo de esto y lo voy a comentar por mientras

In [144]:
# feat_num = len(matches_arr[0,0,:])
# cancha = np.zeros((feat_num, 9, 12), dtype=int)
# cancha[:, top_cord, left_cord] =  matches_arr[0,:,:].T
# cancha[5,:,:]

#### Copio de arriba pero sin T y con 9, 12, feat num

In [145]:
matches_arr[0,:,10]

array([83., 85., 65., 70., 72., 57., 71., 71., 75., 71., 66., 68., 50.,
       60., 73., 68., 50., 59., 70., 68.])

##### Los datos de entrada deben ser tipo (128, 128, 3) -> (pixeles, pixeles, canales de color)

En nuestro caso, (top, left, features).

In [146]:
feat_num = len(matches_arr[0,0,:])
cancha = np.zeros((9, 12, feat_num), dtype=int)
cancha[top_cord, left_cord, :] =  matches_arr[2,:,:]
cancha[:,:,0]

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
###### cancha_stats_lst = []
goles_lst = []
feat_num = len(matches_arr[0,0,:])

for i in range(len(matches_arr[:,0,0])):
    left_cord = matches_arr[i,:,1]
    top_cord = matches_arr[i,:,2]
    goals_arr = matches_arr[i,:,0]
    cancha = np.zeros((feat_num, 9, 12), dtype=int)
    cancha[:, top_cord, left_cord] =  matches_arr[i,:,:].T
    goles_lst.append(cancha[0,:,:])
cancha_stats_lst.append(cancha[3:,:,:])
cancha_stats_lst

In [ ]:
goles_lst

In [ ]:
goles_arr = np.array(goles_lst,int)
goles_arr.shape

In [ ]:
cancha_stats_arr = np.array(cancha_stats_lst, int)
cancha_stats_arr.shape

#### Se obtienen los labels goles_lbls

In [ ]:
goles_lbls = []
#goles_cat_lbl = []
for i in range(cancha_stats_arr[:,0,0,0].size):
   # print(str(i+1) + ' Casa: ' + str(max(goles_arr[i,:,:5].flatten())) 
   #       + '  Visitante: ' + str(max(goles_arr[i,:,6:].flatten())))
    goles_home = max(goles_arr[i,:,:5].flatten())
    goles_away = max(goles_arr[i,:,6:].flatten())
    goles_lbls.append([goles_home, goles_away, goles_home - goles_away, goles_away + goles_home])
    #goles_cat_lbl.append([str(goles_home)+str(goles_away)])
print('goles_home, goles_away, dif_home, goles_tot')
goles_lbls_arr = np.array(goles_lbls)
pd.DataFrame(goles_lbls_arr[:,2]).describe()
#np.unique(goles_cat_lbl)

In [ ]:
goles_dif_df = pd.DataFrame(goles_lbls_arr[:,2])
goles_dif_df.value_counts().sort_index()

In [ ]:
goles_tot_df = pd.DataFrame(goles_lbls_arr[:,3])
goles_tot_df.value_counts().sort_index()

In [ ]:
cancha_stats_arr.shape

In [ ]:
# cancha_stats_arr_inv = cancha_stats_arr.copy()
# print(cancha_stats_arr_inv[0,2,:,:])
print(cancha_stats_arr[0,2,:,:])

### "Volteo" la cancha para tener el doble de instancias

In [ ]:
# cancha_stats_arr_inv = cancha_stats_arr.copy()
# for i in range(12): 
#     cancha_stats_arr_inv[:,:,:,i] = cancha_stats_arr[:,:,:,11-i]
# print(cancha_stats_arr_inv[0,2,:,:])
# print(cancha_stats_arr[0,2,:,:])

In [ ]:
np.unique(goles_lbls_arr[:,0],return_counts=True)

## Finalmenchi

In [ ]:
model = Sequential()

#input_shape=(9,12, 45)
model.add(Conv2D(4, (3,3), padding='same', activation='sigmoid', input_shape=(9,12, 45))) 
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3,3), padding='same', activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (4,4), padding='same', activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(9))


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
cancha_stats_arr.shape

In [ ]:
cancha_stats_arr_inv.shape

In [ ]:
nepe = np.concatenate((cancha_stats_arr, cancha_stats_arr_inv))
nepe.reshape(760, 4860)
Xset_norm = sklearn.preprocessing.normalize(nepe.reshape(760, 4860)).reshape(760,45,9,12)
Xset_norm
a = nepe[0,:,:,:]
a = a.reshape(45,9,12)
a[2,:,:]
np.zeros([2,3,4])

In [ ]:
Yset = np.concatenate((goles_lbls_arr[:,0],goles_lbls_arr[:,1]))

In [ ]:
x_train, x_test, y_train, y_test=train_test_split(Xset_norm, Yset, test_size=0.1, shuffle=True)

epochs = 30
history = model.fit(x_train, y_train, validation_data=(x_test,y_test),  batch_size = 10, epochs = epochs)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

In [ ]:
#train_dataset = dataset.shuffle(len(df)).batch(1)

In [ ]:
#target = df.pop('target')
#dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [ ]:
## get a list of match id
#match_id_ls=matches_df['match_id'].unique().tolist()
## now we can perform a lookup on a 'view' of the dataframe
#dict_match_df = {}

#for match_id in match_id_ls:
    #dict_match_df[match_id] = matches_df[matches_df.match_id==match_id].drop('match_id', axis=1)
#dict_match_df

classes = dict_match_df
for cl in classes:
  img_path = os.path.join(base_dir, cl)
  images = glob.glob(img_path + '/*.jpg')
  print("{}: {} Images".format(cl, len(images)))
  num_train = int(round(len(images)*0.8))
  train, val = images[:num_train], images[num_train:]

  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):
      os.makedirs(os.path.join(base_dir, 'train', cl))
    shutil.move(t, os.path.join(base_dir, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', cl)):
      os.makedirs(os.path.join(base_dir, 'val', cl))
    shutil.move(v, os.path.join(base_dir, 'val', cl)) 

tf.constant(df['thal'])